# Internet of Things: Tiny Machine Learning on Chips.
## 1  Objectives

In this lab, we will explore the techniques and tools to enable artifical intelligence on small embedded devices. We will first learn how to train tiny machine learning (TinyML) models with TensorFlow Lite Microcontroller. We will then explore the methods to quantize and deploy the trained model to a Arduino Nano hardware.

Please send your results (PDF from jupyter) to nhat.pham@cs.ox.ac.uk by Monday.

## 2  Training TinyML models with TensorFlow Lite Microcontrollers

###2.1 Install depedencies. 

In [ ]:
## Create and define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

## Install Dependencies
! pip install tensorflow==2.11.0

## Import dependencies
# TensorFlow is an open source machine learning library
import tensorflow as tf

# Keras is TensorFlow's high-level API for deep learning
from tensorflow import keras
# Numpy is a math library
import numpy as np
# Pandas is a data manipulation library 
import pandas as pd
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math

# Set seed for experiment reproducibility
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

### 2.2 Generate some noisy data.
To build our model, we will generate synthetic data representing a simple simulated heartbeat signal with a fundamental frequency at 0.5Hz and its second-order harmonic at 1Hz. We will also add some noise to our data to better emulate the real world.

In [ ]:
# Number of sample datapoints
SAMPLES = 4000
pi = 3.14159265359
f1 = 0.5 # the fundamental frequency (Hz).
f2 = 2*f1 # the second-order harmonic.

# Generate a uniformly distributed set of random numbers in the range from 0 to 6s.
x_values = np.random.uniform(
    low=0, high=6, size=SAMPLES).astype(np.float32)

# Shuffle the values to guarantee they're not in order
np.random.shuffle(x_values)

# Calculate the corresponding y values
y_values = np.sin(2*pi*f1*x_values).astype(np.float32) + np.sin(2*pi*f2*x_values).astype(np.float32)
# Add a small random number to each y value
y_values += 0.5 * np.random.randn(*y_values.shape)

# Plot our data. The '.' argument tells the library to print blue dots.
plt.plot(x_values, y_values, 'b.')
plt.show()

### 2.3 Split the data.

We now have a noisy dataset representing real-world signals. We will use this data to train a predictor to predict future values. To train the model, we will first need to split the dataset into three sets, i.e., (1) training, (2) validation, and (3) testing sets.

The data is split as follows:
  1. Training: 70%
  2. Validation: 20%
  3. Testing: 10% 

**TODO-1:** Complete the code below to split the data.


In [ ]:
# We'll use 70% of our data for training and 10% for testing. The remaining 20%
# will be used for validation. Calculate the indices of each section.

##### TODO-1: complete the code below
TRAIN_SPLIT =  
TEST_SPLIT = 

# Use np.split to chop our data into three parts.
x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

# Plot the data.
plt.plot(x_train, y_train, 'b.', label="Train")
plt.plot(x_test, y_test, 'r.', label="Test")
plt.plot(x_validate, y_validate, 'y.', label="Validate")
plt.legend()
plt.show()

### 2.4 Training.

Now, let's train a neural network model to predict the signal. Our starting point has a very simple design with only one hidden fully-connected layer with 8 neurons.

In [ ]:
model_1 = tf.keras.Sequential()

# First layer takes a scalar input and feeds it through 8 "neurons". The
# neurons decide whether to activate based on the 'relu' activation function.
model_1.add(keras.layers.Dense(8, activation='relu', input_shape=(1,)))

# Final layer is a single neuron, since we want to output a single value
model_1.add(keras.layers.Dense(1))

# Compile the model using the standard 'adam' optimizer and the mean squared error or 'mse' loss function for regression.
model_1.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model on our training data while validating on our validation set
history_1 = model_1.fit(x_train, y_train, epochs=10, batch_size=64,
                        validation_data=(x_validate, y_validate))

# Calculate and print the loss on our test dataset
test_loss, test_mae = model_1.evaluate(x_test, y_test)

### 2.4 Test the trained model.
After having the trained model, we will test it against our testing data.

In [ ]:
# Make predictions based on our test dataset
y_test_pred = model_1.predict(x_test)

# Graph the predictions against the actual values
plt.clf()
plt.title('Comparison of predictions and actual values')
plt.plot(x_test, y_test, 'b.', label='Actual values')
plt.plot(x_test, y_test_pred, 'r.', label='TF predictions')
plt.legend()
plt.show()

*It turns out that our simple model performs very poorly.*

**TODO-2:** Explain the possible reasons here.

*Your answer goes here.*

### 2.5 Improve the model.
**TODO-3:** Now let's improve our model to predict the signal better. Try to play with different parameters (e.g., number of hidden layers and neurons and training epochs) and complete the code below. Be careful not to overextend the model as it might not be able to deploy on hardware later on (our Arduino Nano has a maximum of 1MB of FLASH and 256KB of RAM). The smaller the model, the easier to deploy and run faster on hardware.

**[Optional]** How can we know which parameter values are optimal for our model training?

In [ ]:
### 2.5 Improve the model.
model = tf.keras.Sequential()

# First layer takes a scalar input and feeds it through the "neurons". The
# neurons decide whether to activate based on the 'relu' activation function.

##### TODO-3: Design and train a new model here

#####

# Save the model to disk
model.save(MODEL_TF)

# Calculate and print the loss on our test dataset
test_loss, test_mae = model.evaluate(x_test, y_test)

# Make predictions based on our test dataset
y_test_pred = model.predict(x_test)

# Graph the predictions against the actual values
plt.clf()
plt.title('Comparison of predictions and actual values')
plt.plot(x_test, y_test, 'b.', label='Actual values')
plt.plot(x_test, y_test_pred, 'r.', label='TF predicted')
plt.legend()
plt.show()

## 3 Generate deployable TensorFlow Lite model

Before we could deploy our model on hardware, we will need to convert it into a deployable TensorFlow Lite model and generate a hardware C code.

### 3.1 TF model generation

The following code will convert the TensorFlow model into the TensorFlow Lite format and save it to the models folder.

In [ ]:
# Convert the model to the TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

### 3.2 Compare model performance

After the conversion, it is always good to compare the performance of the generated and original models to see whether we have any performance drops. The following code will plot the predictions of both models on the testing data.

In [ ]:
def predict_tflite(tflite_model, x_test):
  # Prepare the test data
  x_test_ = x_test.copy()
  x_test_ = x_test_.reshape((x_test.size, 1))
  x_test_ = x_test_.astype(np.float32)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    x_test_ = x_test_ / input_scale + input_zero_point
    x_test_ = x_test_.astype(input_details["dtype"])
  
  # Invoke the interpreter
  y_pred = np.empty(x_test_.size, dtype=output_details["dtype"])
  for i in range(len(x_test_)):
    interpreter.set_tensor(input_details["index"], [x_test_[i]])
    interpreter.invoke()
    y_pred[i] = interpreter.get_tensor(output_details["index"])[0]
  
  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    y_pred = y_pred.astype(np.float32)
    y_pred = (y_pred - output_zero_point) * output_scale

  return y_pred

def evaluate_tflite(tflite_model, x_test, y_true):
  global model
  y_pred = predict_tflite(tflite_model, x_test)
  loss_function = tf.keras.losses.get(model.loss)
  loss = loss_function(y_true, y_pred).numpy()
  return loss

In [ ]:
# Calculate predictions
y_test_pred_tf = model.predict(x_test)
y_test_pred_no_quant_tflite = predict_tflite(model_no_quant_tflite, x_test)

# Compare predictions
plt.clf()
plt.title('Comparison of various models against actual values')
plt.plot(x_test, y_test, 'bo', label='Actual values')
plt.plot(x_test, y_test_pred_tf, 'ro', label='TF predictions')
plt.plot(x_test, y_test_pred_no_quant_tflite, 'bx', label='TFLite predictions')
plt.legend()
plt.show()

# Calculate size
size_tf = os.path.getsize(MODEL_TF)
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)

# Compare size
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes"],
     ["TensorFlow Lite", f"{size_no_quant_tflite} bytes "]],
     columns = ["Model", "Size"], index="Model")

### 3.3 Generate hardware C code.
Now, our model is ready to be deployed. Let's parse the TF Lite model into C code so that we can compile it for the hardware.

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_NO_QUANT_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_NO_QUANT_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

# Print the C source file
!cat {MODEL_TFLITE_MICRO}

## 4 Deploy to Arduino Nano


### 4.1 Arduino Setup
- Install Arduino IDE to your laptop from [here](https://support.arduino.cc/hc/en-us/articles/360019833020-Download-and-install-Arduino-IDE).
- Install `Arduino_TensorFlowLite` library from [here](https://github.com/tensorflow/tflite-micro-arduino-examples#how-to-install).
- Connect to the Arduino board:
  - Open Arduino IDE.
  - Connect Arduino Nano to your laptop with the provided cable.
  - Install `Arduino Mbed OS Nano Boards` library as recommended in Arduino IDE. Otherwise, this library can be found from `Tools -> Manage Libraries...`
  - Choose the correct `Board` and `Port` in `Tools` menu.




### 4.2 Model deployment.
Open lab3.ino in the Arduino folder and go to file model.cpp.
**TODO-4:** Copy the generated C source code of the model and paste it into the model.cpp file. Complile and upload the project to the Arduino. The output signal can be viewed by using the Serial plotter. Please take a screenshot of the output in Serial plotter and place it here.

*Your answer goes here.*

### 4.3 Runtime measurement. 
**TODO-5**: Use the [micros()](https://www.arduino.cc/reference/en/language/functions/time/micros/) function of Arduino to measure the CPU time to run one inference on the Arduino Nano hardware and note down the results here. 

*Your answer goes here.*

### **[Optional]** 4.4 Post-training quantisation.
Runtime performance and model size are essential for on-chip machine learning models.

**TODO-6:** Follow the [tutorial](https://www.tensorflow.org/lite/performance/post_training_quantization) on the TensorFlow website to convert your model to use a quantisation technique of your choice. Rerun steps 3.1-4.3 with the new model and report:
1.   Prediction performance after quantisation vs the original model.
2.   Model size
3. CPU time for each inference

(Please don't modify steps 3.1-4.3 as it will overwrite the old results.)

*Your answer goes here.*

### **[Optional]** 4.5. Other examples.

There are several fun examples to play with the Arduino Nano hardware, such as keyword detection by using microphone signals or gesture recognition with accelerometers, which can be found [here](https://github.com/tensorflow/tflite-micro-arduino-examples).